In [1]:
import os, copy, time
from pathlib import Path

import pandas as pd
import numpy as np

#https://scikit-learn.org/stable/modules/preprocessing.html
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import SimpleImputer

# We will use multiprocessing for simple additive processes. Further, we will expend for other cases.
# Sources: 
#https://medium.com/@vasista/parallel-processing-with-pandas-c76f88963005
#https://towardsdatascience.com/speeding-up-and-perfecting-your-work-using-parallel-computing-8bc2f0c073f8
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

# home-credit-default-risk tables
if not Path("./application_test.csv").is_file():
  ! wget https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0 -O data.zip
  ! unzip -q data.zip

# default-of-credit-card-clients-dataset
if not Path("./default_ucr.csv").is_file():
  ! wget https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0 -O default_ucr.csv

In [2]:
from fcleaning import (EmptyElim,
                       OutlDetect,                       
)
from fencoding import (Dtimetodata,
                       FEncoding,
                       FImputation    
)

In [3]:
# Read in the source datasets
dict = {
    'datasets':[
                # home-credit-default-risk tables
                pd.read_csv('./application_train.csv'),
                pd.read_csv('./application_test.csv'),
                pd.read_csv('./bureau.csv'),
                #pd.read_csv('./bureau_balance.csv'),
                #pd.read_csv('./POS_CASH_balance.csv'),
                #pd.read_csv('./credit_card_balance.csv'),
                # pd.read_csv('./previous_application.csv'),
                # pd.read_csv('./installments_payments.csv'),

                # default-of-credit-card-clients-datasets
                # pd.read_csv('./default_ucr.csv'),
    ],

    'name_dropped_columns':
                [
                 # home-credit-default-risk tables
                 ['SK_ID_CURR', 'TARGET'],
                 ['SK_ID_CURR'],
                 ['SK_ID_CURR',	'SK_ID_BUREAU'],
                 ['SK_ID_BUREAU'],
                 ['SK_ID_PREV',	'SK_ID_CURR'],
                 ['SK_ID_PREV',	'SK_ID_CURR'],
                 ['SK_ID_PREV',	'SK_ID_CURR'],
                 ['SK_ID_PREV',	'SK_ID_CURR'],

                 # default-of-credit-card-clients-datasets
                 ['ID', 'default.payment.next.month']              
    ],   
}

# Keep ID and target columns separately
dict['dropped_columns'] = [dict['datasets'][i][dict['name_dropped_columns'][i]] for i in range(len(dict['datasets']))]

# Drop ID and target columns from the tables
dict['datasets'] = [dict['datasets'][i].drop(dict['name_dropped_columns'][i], axis=1) for i in range(len(dict['datasets']))]

[np.unique([str(dict['datasets'][i][column].dtype) for column in dict['datasets'][i].columns]) for i in range(len(dict['datasets']))]

[array(['float64', 'int64', 'object'], dtype='<U7'),
 array(['float64', 'int64', 'object'], dtype='<U7'),
 array(['float64', 'int64', 'object'], dtype='<U7')]

In [4]:
X=pd.DataFrame([[1,2, np.nan], [3,4, np.nan], [np.nan, np.nan, 2], [10000, 2, np.nan], [0.1, 3,2], [1,1,2]])
X

,0,1,2
0,1.0,2.0,NaN
1,3.0,4.0,NaN
2,NaN,NaN,2.0
3,10000.0,2.0,NaN
4,0.1,3.0,2.0
5,1.0,1.0,2.0


In [5]:
emptyelim = EmptyElim(n_jobs = -1,
                      chunks = None,
                      )
emptyelim.fit(X)


 col_names: {2: [nan, 2.0]}


In [6]:
emptyelim.transform(X)

,0,1
0,1.0,2.0
1,3.0,4.0
2,NaN,NaN
3,10000.0,2.0
4,0.1,3.0
5,1.0,1.0


In [7]:
emptyelim.fit_transform(X)


 col_names: {2: [nan, 2.0]}


,0,1
0,1.0,2.0
1,3.0,4.0
2,NaN,NaN
3,10000.0,2.0
4,0.1,3.0
5,1.0,1.0


In [8]:
outldetect = OutlDetect(outliers_detection_technique='iqr_proximity_rule', 
                        n_jobs=-1)

In [9]:
outldetect.fit(X)


 col_outl_info: {0: (-2.0, 6.0), 1: (0.5, 4.5), 2: (2.0, 2.0)}


In [10]:
outldetect.transform(X)

,0,1,2
0,1.0,2.0,NaN
1,3.0,4.0,NaN
2,NaN,NaN,2.0
3,6.0,2.0,NaN
4,0.1,3.0,2.0
5,1.0,1.0,2.0


In [11]:
outldetect.fit_transform(X)


 col_outl_info: {0: (-2.0, 6.0), 1: (0.5, 4.5), 2: (2.0, 2.0)}


,0,1,2
0,1.0,2.0,NaN
1,3.0,4.0,NaN
2,NaN,NaN,2.0
3,6.0,2.0,NaN
4,0.1,3.0,2.0
5,1.0,1.0,2.0


In [12]:
X = dict['datasets'][2][['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']]
X

,DAYS_CREDIT,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,DAYS_CREDIT_UPDATE
0,-497,-153.0,-153.0,-131
1,-208,1075.0,NaN,-20
2,-203,528.0,NaN,-16
3,-203,NaN,NaN,-16
4,-629,1197.0,NaN,-21
...,...,...,...,...
1716423,-44,-30.0,NaN,-19
1716424,-2648,-2433.0,-2493.0,-2493
1716425,-1809,-1628.0,-970.0,-967
1716426,-1878,-1513.0,-1513.0,-1508


In [13]:
dtimetodata = Dtimetodata(n_jobs=-1, dtime_col_names = {
                                                        'ddays' : ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE'],
                                                        'dmonths' : [],
                                                        'dyears' : []
                                                      },               
                              time_encode = True, 
                              drop_current = True
                              )

In [14]:
dtimetodata.transform(X)

,DAYS_CREDIT_date_year,DAYS_CREDIT_date_month,DAYS_CREDIT_date_day,DAYS_CREDIT_ENDDATE_date_year,DAYS_CREDIT_ENDDATE_date_month,DAYS_CREDIT_ENDDATE_date_day,DAYS_ENDDATE_FACT_date_year,DAYS_ENDDATE_FACT_date_month,DAYS_ENDDATE_FACT_date_day,DAYS_CREDIT_UPDATE_date_year,DAYS_CREDIT_UPDATE_date_month,DAYS_CREDIT_UPDATE_date_day
0,2018,8,22,2019.0,8.0,1.0,2019.0,8.0,1.0,2019,8,23
1,2019,6,7,2022.0,12.0,11.0,NaN,NaN,NaN,2019,12,12
2,2019,6,12,2021.0,6.0,12.0,NaN,NaN,NaN,2019,12,16
3,2019,6,12,NaN,NaN,NaN,NaN,NaN,NaN,2019,12,16
4,2018,4,12,2023.0,4.0,12.0,NaN,NaN,NaN,2019,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,2019,11,18,2019.0,12.0,2.0,NaN,NaN,NaN,2019,12,13
1716424,2012,10,1,2013.0,5.0,4.0,2013.0,3.0,5.0,2013,3,5
1716425,2015,1,18,2015.0,7.0,18.0,2017.0,5.0,6.0,2017,5,9
1716426,2014,11,10,2015.0,11.0,10.0,2015.0,11.0,10.0,2015,11,15


In [15]:
X = dict['datasets'][2]
X

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [16]:
fencoding = FEncoding(n_jobs=-1)
fencoding.initialize_types(X)


 DAYS_CREDIT has type int64 and number of unique values: 2923, will be considered as a numerical 


 CREDIT_DAY_OVERDUE has type int64 and number of unique values: 942, will be considered as a numerical 


 DAYS_CREDIT_UPDATE has type int64 and number of unique values: 2982, will be considered as a numerical 



{'categor_columns': ['CREDIT_ACTIVE',
  'CREDIT_CURRENCY',
  'CNT_CREDIT_PROLONG',
  'CREDIT_TYPE'],
 'numer_columns': ['DAYS_CREDIT',
  'CREDIT_DAY_OVERDUE',
  'DAYS_CREDIT_ENDDATE',
  'DAYS_ENDDATE_FACT',
  'AMT_CREDIT_MAX_OVERDUE',
  'AMT_CREDIT_SUM',
  'AMT_CREDIT_SUM_DEBT',
  'AMT_CREDIT_SUM_LIMIT',
  'AMT_CREDIT_SUM_OVERDUE',
  'DAYS_CREDIT_UPDATE',
  'AMT_ANNUITY'],
 'time_columns': []}

In [17]:
fencoding.encode_categor(X, method = 'OneHotEncoder')


 DAYS_CREDIT has type int64 and number of unique values: 2923, will be considered as a numerical 


 CREDIT_DAY_OVERDUE has type int64 and number of unique values: 942, will be considered as a numerical 


 DAYS_CREDIT_UPDATE has type int64 and number of unique values: 2982, will be considered as a numerical 



,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CNT_CREDIT_PROLONG,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_ACTIVE_nan,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_CURRENCY_nan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,-497,0,-153.0,-153.0,NaN,91323.00,0.0,NaN,0.0,-131,NaN,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,-208,0,1075.0,NaN,NaN,225000.00,171342.0,NaN,0.0,-20,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,-203,0,528.0,NaN,NaN,464323.50,NaN,NaN,0.0,-16,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-203,0,NaN,NaN,NaN,90000.00,NaN,NaN,0.0,-16,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,-629,0,1197.0,NaN,77674.5,2700000.00,NaN,NaN,0.0,-21,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,-44,0,-30.0,NaN,0.0,11250.00,11250.0,0.0,0.0,-19,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1716424,-2648,0,-2433.0,-2493.0,5476.5,38130.84,0.0,0.0,0.0,-2493,NaN,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1716425,-1809,0,-1628.0,-970.0,NaN,15570.00,NaN,NaN,0.0,-967,NaN,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1716426,-1878,0,-1513.0,-1513.0,NaN,36000.00,0.0,0.0,0.0,-1508,NaN,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
X = fencoding.encode_categor(X, method = 'OrdinalEncoder')
X


 DAYS_CREDIT has type int64 and number of unique values: 2923, will be considered as a numerical 


 CREDIT_DAY_OVERDUE has type int64 and number of unique values: 942, will be considered as a numerical 


 DAYS_CREDIT_UPDATE has type int64 and number of unique values: 2982, will be considered as a numerical 



,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE,CREDIT_CURRENCY,CNT_CREDIT_PROLONG,CREDIT_TYPE
0,-497,0,-153.0,-153.0,NaN,91323.00,0.0,NaN,0.0,-131,NaN,2.0,0.0,0.0,3.0
1,-208,0,1075.0,NaN,NaN,225000.00,171342.0,NaN,0.0,-20,NaN,0.0,0.0,0.0,4.0
2,-203,0,528.0,NaN,NaN,464323.50,NaN,NaN,0.0,-16,NaN,0.0,0.0,0.0,3.0
3,-203,0,NaN,NaN,NaN,90000.00,NaN,NaN,0.0,-16,NaN,0.0,0.0,0.0,4.0
4,-629,0,1197.0,NaN,77674.5,2700000.00,NaN,NaN,0.0,-21,NaN,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,-44,0,-30.0,NaN,0.0,11250.00,11250.0,0.0,0.0,-19,NaN,0.0,0.0,0.0,10.0
1716424,-2648,0,-2433.0,-2493.0,5476.5,38130.84,0.0,0.0,0.0,-2493,NaN,2.0,0.0,0.0,3.0
1716425,-1809,0,-1628.0,-970.0,NaN,15570.00,NaN,NaN,0.0,-967,NaN,2.0,0.0,0.0,3.0
1716426,-1878,0,-1513.0,-1513.0,NaN,36000.00,0.0,0.0,0.0,-1508,NaN,2.0,0.0,0.0,3.0


In [19]:
X = fencoding.bucket_numerical(X, n_bins=5, columns_to_buck = 'all_numerical', 
                           drop_current = True)


 DAYS_CREDIT has type int64 and number of unique values: 2923, will be considered as a numerical 


 CREDIT_DAY_OVERDUE has type int64 and number of unique values: 942, will be considered as a numerical 


 DAYS_CREDIT_UPDATE has type int64 and number of unique values: 2982, will be considered as a numerical 


 CREDIT_ACTIVE has type float64 and number of unique values: 4, will be considered as a categorical 


 CREDIT_CURRENCY has type float64 and number of unique values: 4, will be considered as a categorical 


 CREDIT_TYPE has type float64 and number of unique values: 15, will be considered as a categorical 


 CNT_CREDIT_PROLONG has type float64 and number of unique values: 10, will be considered as a categorical 


 DAYS_CREDIT bucketing ...

 AMT_CREDIT_SUM_LIMIT bucketing ...

 CREDIT_DAY_OVERDUE bucketing ...

 AMT_CREDIT_SUM_OVERDUE bucketing ...

 DAYS_CREDIT_ENDDATE bucketing ...

 DAYS_CREDIT_UPDATE bucketing ...

 DAYS_ENDDATE_FACT bucketing ...

 AMT_ANNUITY bucketing 

In [20]:
X

,DAYS_CREDIT_bucketed,CREDIT_DAY_OVERDUE_bucketed,DAYS_CREDIT_ENDDATE_bucketed,DAYS_ENDDATE_FACT_bucketed,AMT_CREDIT_MAX_OVERDUE_bucketed,AMT_CREDIT_SUM_bucketed,AMT_CREDIT_SUM_DEBT_bucketed,CREDIT_ACTIVE,CREDIT_CURRENCY,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM_LIMIT_bucketed,AMT_CREDIT_SUM_OVERDUE_bucketed,DAYS_CREDIT_UPDATE_bucketed,AMT_ANNUITY_bucketed,CREDIT_TYPE
0,3,1,5,5,0,5,5,2.0,0.0,0.0,0,1,3,0,3.0
1,4,1,5,0,0,5,5,0.0,0.0,0.0,0,1,4,0,4.0
2,4,1,5,0,0,5,0,0.0,0.0,0.0,0,1,4,0,3.0
3,4,1,0,0,0,5,0,0.0,0.0,0.0,0,1,4,0,4.0
4,3,1,5,0,5,5,0,0.0,0.0,0.0,0,1,4,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,4,1,5,0,5,5,5,0.0,0.0,0.0,5,1,4,0,10.0
1716424,1,1,5,5,5,5,5,2.0,0.0,0.0,5,1,1,0,3.0
1716425,1,1,5,5,0,5,0,2.0,0.0,0.0,0,1,1,0,3.0
1716426,1,1,5,5,0,5,5,2.0,0.0,0.0,5,1,1,0,3.0


In [21]:
fimputation = FImputation('tree-based', 
                          fill_with_value = 'extreme_values', 
                          n_jobs = -1,
                          )
fimputation.impute(X)

,DAYS_CREDIT_bucketed,CREDIT_DAY_OVERDUE_bucketed,DAYS_CREDIT_ENDDATE_bucketed,DAYS_ENDDATE_FACT_bucketed,AMT_CREDIT_MAX_OVERDUE_bucketed,AMT_CREDIT_SUM_bucketed,AMT_CREDIT_SUM_DEBT_bucketed,CREDIT_ACTIVE,CREDIT_CURRENCY,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM_LIMIT_bucketed,AMT_CREDIT_SUM_OVERDUE_bucketed,DAYS_CREDIT_UPDATE_bucketed,AMT_ANNUITY_bucketed,CREDIT_TYPE
0,3,1,5,5,0,5,5,2.0,0.0,0.0,0,1,3,0,3.0
1,4,1,5,0,0,5,5,0.0,0.0,0.0,0,1,4,0,4.0
2,4,1,5,0,0,5,0,0.0,0.0,0.0,0,1,4,0,3.0
3,4,1,0,0,0,5,0,0.0,0.0,0.0,0,1,4,0,4.0
4,3,1,5,0,5,5,0,0.0,0.0,0.0,0,1,4,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,4,1,5,0,5,5,5,0.0,0.0,0.0,5,1,4,0,10.0
1716424,1,1,5,5,5,5,5,2.0,0.0,0.0,5,1,1,0,3.0
1716425,1,1,5,5,0,5,0,2.0,0.0,0.0,0,1,1,0,3.0
1716426,1,1,5,5,0,5,5,2.0,0.0,0.0,5,1,1,0,3.0
